In [25]:
#Only needed when using Google Colab

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir(r"/content/drive/My Drive/Predictive Analytics/Francisco/Combined model")
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 assets.pkl	        emotions_model.keras   sentiment_model.keras
 Combined_model.ipynb   emotion_utils.py       tokenizer.pkl
 custom_funtions.py     labels_2.csv	       Try_using_function.ipynb
'Demo Diary.ipynb'      __pycache__


# Import models

In [26]:
# Imports
import os
import pandas as pd
import pickle
from datetime import datetime
from keras.models import load_model
from emotion_utils import predict_sent_emotions
from custom_funtions import LearnablePositionalEncoding, focal_loss_with_penalty, f1_metric

# File paths
CSV_FILE = "mood_diary.csv" #if run in colab the whole path needs to be added e.g. /content/drive/My Drive/Predictive Analytics/..

# Load your custom models
model_emotion = load_model("emotions_model.keras", custom_objects={
    "LearnablePositionalEncoding": LearnablePositionalEncoding,
    "focal_loss_with_penalty": focal_loss_with_penalty,
    "f1_metric": f1_metric
})

model_sentiment = load_model("sentiment_model.keras")

# Load tokenizer and assets
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("assets.pkl", "rb") as f:
    data = pickle.load(f)
    word_index = data["word_index"]
    emotion_columns = data["emotion_columns"]


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 24 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Set up Diary

In [36]:
# Ensure diary CSV exists
if not os.path.exists(CSV_FILE):
    pd.DataFrame(columns=["date", "entry", "mood", "feeling"]).to_csv(CSV_FILE, index=False)

# Load current diary
df = pd.read_csv(CSV_FILE)

# Motivational messages
positive_messages = [
    "You're doing amazing – keep it up! 🌟",
    "What a great vibe today! ✨",
    "Keep spreading the positivity! 💖",
    "That's the spirit! Stay awesome 😎",
    "Positive days build strong futures 🚀"
]

negative_messages = [
    "Tough times don’t last. You do. 💪",
    "You're stronger than you feel right now. 🧡",
    "Breathe. You've got this. 🌈",
    "It's okay to feel down sometimes. You're not alone. 🤗",
    "One step at a time. You’re doing great. 🌱"
]

# Helper function
def classify_mood_and_feeling(prediction):
    mood_label = prediction[0][0]  # "Positive" or "Negative"
    feeling_label = prediction[0][1][0]  # top emotion
    return mood_label, feeling_label


#Weekly summary funtion
def show_weekly_summary(df, current_date):
    current_date = pd.to_datetime(current_date)
    if current_date.weekday() != 6:  # Only on Sundays
        return df

    one_week_ago = current_date - pd.Timedelta(days=6)
    df["date_parsed"] = pd.to_datetime(df["date"])
    last_week_df = df[
        (df["date_parsed"] >= one_week_ago) &
        (df["date_parsed"] <= current_date) &
        (df["entry"] != "WEEKLY SUMMARY")  # Exclude past summaries
    ]

    mood_counts = last_week_df["mood"].value_counts()
    positive_days = mood_counts.get("Positive", 0)
    negative_days = mood_counts.get("Negative", 0)

    print("\n📊 Weekly Mood Summary:")
    print(f"✔️ Positive days: {positive_days}")
    print(f"❌ Negative days: {negative_days}")

    if negative_days > positive_days:
        summary_mood = "More Negative"
        feeling_summary = "You've had a tough week"
        print("\n🚨 You've had a tough week.")
        print("🧡 Recommendation: Talk to someone you trust, take a walk, and take time for yourself.")
    else:
        summary_mood = "More Positive"
        feeling_summary = "Keep up the positive momentum"
        print("\n✅ Keep up the positive momentum! You're doing great.")

    # Add summary row
    summary_row = {
        "date": current_date.strftime("%Y-%m-%d"),
        "entry": "WEEKLY SUMMARY",
        "mood": summary_mood,
        "feeling": feeling_summary
    }
    df = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)
    return df



## Write diary entry

In [44]:
# Get user input
entry = input("📝 Write your diary entry: ")
date_input = input("📅 Enter date (YYYY-MM-DD) or press Enter for today: ")

if date_input.strip() == "":
    date = datetime.now().strftime("%Y-%m-%d")
else:
    date = date_input.strip()

# Predict using your custom models
prediction = predict_sent_emotions(entry, model_emotion, model_sentiment, tokenizer,
                                   word_index, emotion_columns, max_len=100, threshold=0.5)

# Extract mood and feeling
mood, feeling = classify_mood_and_feeling(prediction)

# Motivational response
if mood == "Positive":
    print(f"\n😊 Mood detected: {mood} ({feeling})")
    print("💬", pd.Series(positive_messages).sample(1).values[0])
else:
    print(f"\n😟 Mood detected: {mood} ({feeling})")
    print("💬", pd.Series(negative_messages).sample(1).values[0])

# Save to diary:
new_entry = {"date": date, "entry": entry, "mood": mood, "feeling": feeling}
df = df[df["date"] != date]
df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)

# Call summary function and update df if needed
df = show_weekly_summary(df, date)

# Sort and save
df = df.sort_values("date").reset_index(drop=True)
df.to_csv(CSV_FILE, index=False)

# Display diary
from IPython.display import display
display(df)




📝 Write your diary entry: I am very curious about my Predictive Analytics exam tomorrow. I just hope the professor is as nice as he always is :)	
📅 Enter date (YYYY-MM-DD) or press Enter for today: 2025-05-25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

😊 Mood detected: Positive (optimism)
💬 Positive days build strong futures 🚀

📊 Weekly Mood Summary:
✔️ Positive days: 5
❌ Negative days: 2

✅ Keep up the positive momentum! You're doing great.


,date,entry,mood,feeling,date_parsed
0,2025-05-12,i had a terrible tennis match today.,Negative,fear,2025-05-12
1,2025-05-13,I am disappointed! The food in the canteen was...,Negative,disgust,2025-05-13
2,2025-05-14,i had a sad time today! All my friends did not...,Positive,sadness,2025-05-14
3,2025-05-15,Amazing experiences i had today! Had a run to ...,Positive,admiration,2025-05-15
4,2025-05-16,It was a stressful day. We had a deadline for ...,Negative,neutral,2025-05-16
5,2025-05-17,What a cool time i had today! Went to a footba...,Positive,excitement,2025-05-17
6,2025-05-18,This day was a horrible experience. I was almo...,Negative,fear,2025-05-18
7,2025-05-18,WEEKLY SUMMARY,More Negative,You've had a tough week,2025-05-18
8,2025-05-19,Terrible day! I was so tired and did not have ...,Negative,neutral,2025-05-19
9,2025-05-20,It was a tough day. I spilled my tea all over ...,Positive,neutral,2025-05-20
